#  Equity Valuation Demo

This notebook demonstrates multiple equity valuation models in Python using `yfinance`.  
It supports automatic data fetching, model calculation, and export of results in CSV and HTML (with colour-coded valuation notes).

---

## 🧮 Implemented & Supported Models

| Model | Status | Data Source / Calculation | Formula |
|---|---|---|---|
| **DCF (Discounted Cash Flow)** | ✅ Implemented | Free cash flow from cashflow statement | See formula below |
| **DDM (Dividend Discount Model)** | ✅ Implemented | Dividend history or EPS × payout ratio | Two-stage growth |
| **Graham Number** | ✅ Implemented | EPS & Book Value per Share | $$ \sqrt{22.5 \times EPS \times BVPS} $$ |
| **Buffett Fair Value Formula** | ✅ Implemented | EPS & growth rate | $$ EPS \times (8.5 + 2 \times g) $$ |
| **Relative Valuation (P/E, P/B)** | ⚠️ Data pulled only | `trailingPE` & `priceToBook` from `yfinance` | Direct comparison |
| **Residual Income Model (ROE)** | ⚠️ Data pulled only | `returnOnEquity`, BVPS, cost of equity | $$ BV + \sum \frac{RI_t}{(1+r)^t} $$ |
| **EV/EBITDA Multiple** | ⚠️ Data pulled only | EV & EBITDA | $$ \frac{EV}{EBITDA} $$ |

✅ = fully calculated in code • ⚠️ = data retrieved; extend as needed

---

##  Workflow
1. **Load configuration and tickers** – `config.json`, `tickers.txt`  
2. **Fetch financial data** – cash flow, EPS, BVPS, dividends, P/E, P/B, ROE, EV/EBITDA  
3. **Run valuation models** – DCF, DDM, Graham, Buffett  
4. **Aggregate** – median fair value, Upside %, valuation flag  
5. **Export** – `output/results.csv`, `output/report.html`

---

##  Models & Formulae

### 1. Discounted Cash Flow (DCF)

Assumptions:
- Current free cash flow \( FCF_0 \)
- Annual growth rate \( g \)
- Discount rate \( r \)
- Terminal growth rate \( g_t \)
- Projection horizon \( N \)

# Corporate Valuation Formula

## Firm Value Calculation Formula

$$
\text{Firm Value} = \sum_{t=1}^{N} \frac{FCF_0 \cdot (1+g)^t}{(1+r)^t} + \frac{FCF_N \cdot (1+g_t)}{(r - g_t)} \cdot \frac{1}{(1+r)^N}
$$

where: `FCF_N = FCF_0 × (1+g)^N`

## Fair Value per Share

$$
\text{Fair Value per Share} = \frac{\text{Firm Value} + \text{Cash} - \text{Debt}}{\text{Shares Outstanding}}
$$

## Variables

- \( FCF_0 \): base free cash flow
- \( g \): growth rate
- \( r \): discount rate (WACC)
- \( N \): projection years
- \( g_t \): terminal growth rate

---

### 2. Dividend Discount Model (Two-Stage DDM)

Assumptions:
- Current dividend \( D_0 \)
- First-stage growth \( g_1 \) (for \( n \) years)
- Terminal (second-stage) growth \( g_2 \)
- Discount rate \( r \)

Formula:

$$\text{Value} = \sum_{t=1}^{n} \frac{D_0 \times (1+g_1)^t}{(1+r)^t} + \frac{D_n \times (1+g_2)}{(r - g_2)} \times \frac{1}{(1+r)^n}$$

where:

$$D_n = D_0 \times (1+g_1)^n$$

---

### 3. Graham Number

Benjamin Graham’s conservative fair-value formula:

$$
\text{Graham Number} = \sqrt{22.5 \cdot EPS \cdot BVPS}
$$

- **EPS**: earnings per share
- **BVPS**: book value per share

---

### 4. Buffett Approximation

Buffett Fair Value Formula:

$$
\text{Fair Price} = EPS \times (8.5 + 2 \times g)
$$

- \( g \): growth rate (%)

---

> Notes (implementation details in this notebook):
> - DCF requires your own cash flow forecasting; here we approximate using the latest FCF and a growth assumption.
> - DDM takes dividend history when available; otherwise uses EPS × payout ratio as a proxy for \( D_0 \).
> - Buffett method compares your fair price to current market price.
> - Relative valuation (P/E, P/B), ROE-based residual income, and EV/EBITDA are retrieved for comparison; extend formulas as needed.

# 1. DCF model（Discounted Cash Flow）

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import time

def dcf_valuation(ticker, fcf_growth_rate=0.06, discount_rate=0.09, projection_years=5, terminal_growth=0.025):
    stock = yf.Ticker(ticker)
    
    try:
        cashflow = stock.cashflow
        if cashflow.empty:
            return {"Ticker": ticker, "Fair Value": "N/A", "Note": "Cashflow empty"}

        # 正確的自由現金流組成項目
        cf_index = cashflow.index.tolist()
        if "Operating Cash Flow" in cf_index and "Capital Expenditure" in cf_index:
            op_cashflow = cashflow.loc["Operating Cash Flow"].dropna().iloc[-1]
            capex = cashflow.loc["Capital Expenditure"].dropna().iloc[-1]
        else:
            return {"Ticker": ticker, "Fair Value": "N/A", "Note": "Missing FCF components"}

        fcf = float(op_cashflow + capex)
        if fcf <= 0:
            return {"Ticker": ticker, "Fair Value": "N/A", "Note": "FCF <= 0"}

        # 預估未來 FCF 並折現
        fcf_list = [
            fcf * ((1 + fcf_growth_rate) ** year) / ((1 + discount_rate) ** year)
            for year in range(1, projection_years + 1)
        ]

        # 終值與折現
        terminal_fcf = fcf * ((1 + fcf_growth_rate) ** projection_years)
        terminal_value = terminal_fcf * (1 + terminal_growth) / (discount_rate - terminal_growth)
        terminal_value_discounted = terminal_value / ((1 + discount_rate) ** projection_years)

        enterprise_value = sum(fcf_list) + terminal_value_discounted

        # 股權價值估算
        info = stock.info
        debt = info.get("totalDebt", 0) or 0
        cash = info.get("totalCash", 0) or 0
        shares = info.get("sharesOutstanding", 1)
        current_price = info.get("currentPrice", np.nan)

        equity_value = enterprise_value + cash - debt
        fair_value_per_share = equity_value / shares
        margin_of_safety = (fair_value_per_share - current_price) / fair_value_per_share * 100 if current_price else np.nan

        note = "Undervalued" if fair_value_per_share > current_price else "Overvalued"

        return {
            "Ticker": ticker,
            "Current Price ($)": round(current_price, 2),
            "Fair Value per Share ($)": round(fair_value_per_share, 2),
            "Margin of Safety (%)": round(margin_of_safety, 2),
            "Current FCF (M)": round(fcf / 1e6, 2),
            "Note": note
        }

    except Exception as e:
        return {"Ticker": ticker, "Fair Value": "N/A", "Note": str(e)}


tickers =  ["AAPL", "MSFT", "PG", "KO", "JNJ", "AXP", "JPM", "WMT", "XOM", "GOOG", "TSM"]
results = []

for tkr in tickers:
    result = dcf_valuation(tkr)
    results.append(result)
    time.sleep(1) 


df_dcf = pd.DataFrame(results)
df_dcf = df_dcf[df_dcf["Note"].isin(["Undervalued", "Overvalued"])]  # 排除 error 結果
df_dcf = df_dcf.sort_values(by="Margin of Safety (%)", ascending=False).reset_index(drop=True)


pd.set_option('display.float_format', '${:,.2f}'.format)
df_dcf= df_dcf.drop(columns=["Fair Value"], errors="ignore")
df_dcf

,Ticker,Current Price ($),Fair Value per Share ($),Margin of Safety (%),Current FCF (M),Note
0,TSM,$241.83,"$1,254.04",$80.72,"$262,724.30",Undervalued
1,XOM,$106.80,$149.13,$28.38,"$36,053.00",Undervalued
2,GOOG,$202.16,$235.90,$14.31,"$67,012.00",Undervalued
3,AXP,$297.43,$340.44,$12.63,"$13,095.00",Undervalued
4,JNJ,$173.33,$137.04,$-26.48,"$19,758.00",Overvalued
5,PG,$153.51,$94.58,$-62.31,"$13,567.00",Overvalued
6,KO,$70.34,$39.57,$-77.76,"$11,258.00",Overvalued
7,AAPL,$229.35,$111.61,$-105.49,"$92,953.00",Overvalued
8,MSFT,$522.04,$158.17,$-230.04,"$65,149.00",Overvalued
9,WMT,$103.73,$17.98,$-476.94,"$11,075.00",Overvalued


# 2.Dividend Discount Model（DDM）股利折現模型 

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

def ddm_two_stage_auto(ticker, rf=0.045, rm_rf=0.055, n=5, g_default=0.04):
    stock = yf.Ticker(ticker)
    info = stock.info

    try:
        D0 = info.get("dividendRate")
        beta = info.get("beta", 1)
        price = info.get("currentPrice", np.nan)

        # 跳過無股利公司
        if D0 is None or D0 == 0:
            return {"Ticker": ticker, "Fair Value ($)": "N/A", "Note": "No dividend"}

        # Cost of Equity
        r = rf + beta * rm_rf

        
        # 成長率 g 來源穩定，來自分析師預估
        g = info.get("earningsGrowth")
        if g is None or g <= 0:
            g = g_default
        g = min(max(g, 0.02), 0.08)
        
        # 🚨 修正：防止 g ≥ r（終值公式會失效）
        if g >= r:
            g = r - 0.005  # 強制小於 r 一點點
        
    
        
        # DDM 計算
        D_list = [D0 * (1 + g) ** t / (1 + r) ** t for t in range(1, n + 1)]
        terminal_D = D0 * (1 + g) ** n
        terminal_value = terminal_D / (r - g)
        present_terminal = terminal_value / ((1 + r) ** n)
        fair_value = sum(D_list) + present_terminal

        margin = (fair_value - price) / fair_value * 100 if price else np.nan
        note = "Undervalued" if fair_value > price else "Overvalued"

        return {
            "Ticker": ticker,
            "Dividend ($)": round(D0, 2),
            "Beta": round(beta, 2),
            "r (COE)": round(r, 4),
            "g (Growth)": round(g, 4),
            "Fair Value ($)": round(fair_value, 2),
            "Current Price ($)": round(price, 2),
            "Margin of Safety (%)": round(margin, 2),
            "Note": note
        }

    except Exception as e:
        return {"Ticker": ticker, "Fair Value ($)": "N/A", "Note": str(e)}


tickers = ["KO", "JNJ", "PG", "T", "XOM", "AXP", "JPM", "WMT", "TMUS"]

results = [ddm_two_stage_auto(tkr) for tkr in tickers]
df_ddm = pd.DataFrame(results)

# 排除錯誤資料並依安全邊際排序
df_ddm = pd.DataFrame(results).sort_values(by="Margin of Safety (%)", ascending=False).reset_index(drop=True)

pd.set_option('display.float_format', '${:,.2f}'.format)
df_ddm

,Ticker,Dividend ($),Beta,r (COE),g (Growth),Fair Value ($),Current Price ($),Margin of Safety (%),Note
0,T,$1.11,$0.62,$0.08,$0.07,$222.38,$28.08,$87.37,Undervalued
1,JNJ,$5.20,$0.40,$0.07,$0.06,"$1,041.50",$173.33,$83.36,Undervalued
2,KO,$2.04,$0.44,$0.07,$0.06,$408.61,$70.34,$82.79,Undervalued
3,PG,$4.23,$0.37,$0.07,$0.06,$847.19,$153.51,$81.88,Undervalued
4,TMUS,$3.52,$0.61,$0.08,$0.07,$705.19,$244.98,$65.26,Undervalued
5,XOM,$3.96,$0.50,$0.07,$0.04,$122.13,$106.80,$12.55,Undervalued
6,JPM,$5.60,$1.11,$0.11,$0.04,$85.90,$288.76,$-236.18,Overvalued
7,WMT,$0.94,$0.66,$0.08,$0.04,$22.80,$103.73,$-354.94,Overvalued
8,AXP,$3.28,$1.28,$0.12,$0.04,$43.92,$297.43,$-577.21,Overvalued


# 3. Buffett DCF

In [3]:
import yfinance as yf
import numpy as np
import pandas as pd

# CAPM 計算動態折現率
def get_discount_rate(info, rf=0.045, market_premium=0.055):
    beta = info.get("beta", 1.0)
    return rf + beta * market_premium

# Buffett 模型（動態折現率 + 真實配息率）
def buffett_fair_price(eps, roe, payout_ratio, discount_rate, years=10):
    if eps is None or roe is None:
        return None
    # 修正：ROE 必須除以 100
    g = (roe / 100) * (1 - payout_ratio)
    eps_future = eps * ((1 + g) ** years)
    pe = 20 if roe >= 30 else 15 if roe >= 20 else 10 if roe >= 10 else 8
    fair_price = eps_future * pe / ((1 + discount_rate) ** years)
    return fair_price


# 股票清單
tickers = [
    "AAPL",   # Apple - 高 ROE，EPS 穩定
    "MSFT",   # Microsoft - 高 ROE，現金流強
    "KO",     # Coca-Cola - 巴菲特愛股，EPS 穩定
    "PG",     # Procter & Gamble - 長期穩健
    "JNJ",    # Johnson & Johnson - 醫療龍頭，ROE 穩定
    "WMT",    # Walmart - 零售穩健 EPS 成長
    "JPM",    # JPMorgan Chase - 銀行龍頭，EPS 穩定
    "AXP",    # American Express - 巴菲特長期持有
    "TSM",    # 台積電 ADR - 穩定 EPS 與 ROE
]


results = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    info = stock.info

    # 折現率
    discount_rate = get_discount_rate(info)

    # 取得估值參數
    eps = info.get("trailingEps")
    roe = info.get("returnOnEquity")
    price = info.get("currentPrice")
    dividend = info.get("dividendRate")

    # 真實配息比率（若無配息則為 0）
    payout_ratio = max(0, min(1, dividend / eps)) if eps and dividend else 0

    # 巴菲特估值
    buffett_val = buffett_fair_price(eps, roe, payout_ratio, discount_rate=discount_rate)
    buffett_undervalued = (buffett_val is not None and price is not None and price < buffett_val)

    # 結果儲存
    results.append({
        "Ticker": ticker,
        "Current Price": round(price, 2) if price else "N/A",
        "EPS": round(eps, 2) if eps else "N/A",
        "ROE (%)": round(roe * 100, 2) if roe else "N/A",
        "Payout Ratio": round(payout_ratio, 2),
        "r (COE)": round(discount_rate, 4),
        "Margin of Safety (%)": round((buffett_val - price) / buffett_val * 100, 2) if buffett_val else "N/A",
        "Buffett Fair Price ($)": round(buffett_val, 2) if buffett_val else "N/A",
        "Undervalued (Buffett)": "Yes" if buffett_undervalued else "No"
    })

# 整理成表格
df_buffett = pd.DataFrame(results)
df_buffett = df_buffett.sort_values("Undervalued (Buffett)", ascending=False).reset_index(drop=True)
pd.set_option('display.float_format', '${:,.2f}'.format)
df_buffett

,Ticker,Current Price,EPS,ROE (%),Payout Ratio,r (COE),Margin of Safety (%),Buffett Fair Price ($),Undervalued (Buffett)
0,AAPL,$229.35,$6.58,$149.81,$0.16,$0.11,$-982.32,$21.19,No
1,MSFT,$522.04,$13.63,$33.28,$0.24,$0.10,"$-1,144.66",$41.94,No
2,KO,$70.34,$2.82,$42.37,$0.72,$0.07,$-500.75,$11.71,No
3,PG,$153.51,$6.51,$31.24,$0.65,$0.07,$-449.94,$27.91,No
4,JNJ,$173.33,$9.34,$30.21,$0.56,$0.07,$-337.57,$39.61,No
5,WMT,$103.73,$2.34,$21.78,$0.40,$0.08,"$-1,097.53",$8.66,No
6,JPM,$288.76,$19.49,$16.21,$0.29,$0.11,$-400.86,$57.65,No
7,AXP,$297.43,$14.25,$32.81,$0.23,$0.12,$-659.41,$39.17,No
8,TSM,$241.83,$8.34,$34.56,$0.39,$0.11,$-914.47,$23.84,No


# 4. 相對估值模型（P/E, P/B）

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import difflib

# 自動模糊匹配產業名稱
def match_industry_name(industry_name, benchmark_dict):
    candidates = benchmark_dict.keys()
    match = difflib.get_close_matches(industry_name, candidates, n=1, cutoff=0.6)
    return match[0] if match else None

#  產業估值基準資料（來源整合自 NYU Stern, Finbox, Macrotrends）
industry_benchmarks = {
    "Consumer Electronics": {"avg_pe": 22.0, "avg_pb": 6.0},
    "Banks - Diversified": {"avg_pe": 13.8, "avg_pb": 1.2},
    "Household & Personal Products": {"avg_pe": 21.4, "avg_pb": 4.5},
    "Semiconductors": {"avg_pe": 22.3, "avg_pb": 6.8},
    "Software - Infrastructure": {"avg_pe": 30.5, "avg_pb": 10.2},
    "Software - Application": {"avg_pe": 32.1, "avg_pb": 9.5},
    "Medical Devices": {"avg_pe": 28.7, "avg_pb": 5.6},
    "Pharmaceuticals": {"avg_pe": 17.5, "avg_pb": 4.1},
    "Biotechnology": {"avg_pe": 21.0, "avg_pb": 5.2},
    "Oil & Gas Integrated": {"avg_pe": 9.1, "avg_pb": 1.6},
    "Utilities - Regulated Electric": {"avg_pe": 16.2, "avg_pb": 2.0},
    "REIT - Retail": {"avg_pe": 14.1, "avg_pb": 1.5},
    "Insurance - Diversified": {"avg_pe": 11.6, "avg_pb": 1.1},
    "Insurance - Life": {"avg_pe": 8.9, "avg_pb": 0.9},
    "Insurance - Property & Casualty": {"avg_pe": 10.4, "avg_pb": 1.3},
    "Retail - Defensive": {"avg_pe": 20.0, "avg_pb": 4.3},
    "Retail - Cyclical": {"avg_pe": 24.5, "avg_pb": 6.1},
    "Restaurants": {"avg_pe": 27.8, "avg_pb": 8.4},
    "Telecom Services": {"avg_pe": 12.9, "avg_pb": 1.8},
    "Aerospace & Defense": {"avg_pe": 19.2, "avg_pb": 3.4},
    "Industrial Products": {"avg_pe": 18.5, "avg_pb": 2.9},
    "Packaging & Containers": {"avg_pe": 17.8, "avg_pb": 2.5},
    "Auto Manufacturers": {"avg_pe": 11.4, "avg_pb": 1.1},
    "Specialty Chemicals": {"avg_pe": 20.3, "avg_pb": 2.7},
    "Gold": {"avg_pe": 16.7, "avg_pb": 1.9},
    "Banks - Regional": {"avg_pe": 11.2, "avg_pb": 1.0},
    "REIT - Industrial": {"avg_pe": 15.8, "avg_pb": 1.8},
    "Healthcare Plans": {"avg_pe": 17.2, "avg_pb": 3.1},
    "Beverages - Non-Alcoholic": {"avg_pe": 23.1, "avg_pb": 5.5},
    "Internet Retail": {"avg_pe": 61.2, "avg_pb": 15.4},
    "Internet Content & Information": {"avg_pe": 38.7, "avg_pb": 10.3},
    "Credit Services": {"avg_pe": 21.5, "avg_pb": 6.7},
    "Discount Stores": {"avg_pe": 24.0, "avg_pb": 5.0}
}


tickers= [
    "AAPL", "MSFT", "PG", "KO", "AMZN", "META", "NVDA", "AVGO", "AXP", "TSLA",
    "JPM", "WMT", "GOOG", "TSM", "INTC", "CRM"
]


# 建立估值資料結果
relative_results = []

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        pe = info.get("trailingPE", np.nan)
        pb = info.get("priceToBook", np.nan)
        eps = info.get("trailingEps", np.nan)
        industry = info.get("industry", "Unknown")
        sector = info.get("sector", "Unknown")

        matched_industry = match_industry_name(industry, industry_benchmarks)
        industry_avg = industry_benchmarks.get(matched_industry, {"avg_pe": np.nan, "avg_pb": np.nan})

        pe_undervalued = pe < industry_avg["avg_pe"] if not np.isnan(pe) else False
        pb_undervalued = pb < industry_avg["avg_pb"] if not np.isnan(pb) else False
        is_loss_making = eps < 0 if not np.isnan(eps) else False

        relative_results.append({
            "Ticker": ticker,
            "P/E": round(pe, 2) if not np.isnan(pe) else "N/A",
            "P/B": round(pb, 2) if not np.isnan(pb) else "N/A",
            "EPS": round(eps, 2) if not np.isnan(eps) else "N/A",
            "Loss-Making": is_loss_making,
            "Industry": industry,
            "Matched Industry": matched_industry or "N/A",
            "Sector": sector,
            "Industry Avg P/E": industry_avg["avg_pe"],
            "Industry Avg P/B": industry_avg["avg_pb"],
            "P/E Undervalued": pe_undervalued,
            "P/B Undervalued": pb_undervalued
        })
    except Exception as e:
        relative_results.append({
            "Ticker": ticker,
            "Error": str(e)
        })

df_relative = pd.DataFrame(relative_results)
pd.set_option('display.float_format', '{:.2f}'.format)
display(df_relative)


,Ticker,P/E,P/B,EPS,Loss-Making,Industry,Matched Industry,Sector,Industry Avg P/E,Industry Avg P/B,P/E Undervalued,P/B Undervalued
0,AAPL,34.86,51.76,6.58,False,Consumer Electronics,Consumer Electronics,Technology,22.00,6.00,False,False
1,MSFT,38.30,11.30,13.63,False,Software - Infrastructure,Software - Infrastructure,Technology,30.50,10.20,False,False
2,PG,23.58,7.02,6.51,False,Household & Personal Products,Household & Personal Products,Consumer Defensive,21.40,4.50,False,False
3,KO,24.94,10.59,2.82,False,Beverages - Non-Alcoholic,Beverages - Non-Alcoholic,Consumer Defensive,23.10,5.50,False,False
4,AMZN,33.89,7.11,6.57,False,Internet Retail,Internet Retail,Consumer Cyclical,61.20,15.40,True,True
5,META,27.93,9.92,27.54,False,Internet Content & Information,Internet Content & Information,Communication Services,38.70,10.30,True,True
6,NVDA,58.76,53.15,3.11,False,Semiconductors,Semiconductors,Technology,22.30,6.80,False,False
7,AVGO,111.30,5.15,2.74,False,Semiconductors,Semiconductors,Technology,22.30,6.80,False,True
8,AXP,20.87,6.41,14.25,False,Credit Services,Credit Services,Financial Services,21.50,6.70,True,True
9,TSLA,196.24,13.75,1.68,False,Auto Manufacturers,Auto Manufacturers,Consumer Cyclical,11.40,1.10,False,False


# 5.格雷厄姆數（Graham Number）

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np

#  Graham Number
def graham_number(eps, bvps):
    try:
        return np.sqrt(22.5 * eps * bvps) if eps > 0 and bvps > 0 else np.nan
    except:
        return np.nan

tickers = [
    "KO", "PG", "JNJ",  # 成熟型消費與醫療
    "JPM", "WMT", "T",  # 銀行、零售、電信
    "XOM", "AXP", "INTC"  # 能源、金融、半導體（偏成熟）
]



graham_results = []

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        eps = info.get("trailingEps", np.nan)
        bvps = info.get("bookValue", np.nan)
        price = info.get("currentPrice", np.nan)
        name = info.get("shortName", ticker)

        graham_val = graham_number(eps, bvps)
        undervalued = price < graham_val if not np.isnan(graham_val) and not np.isnan(price) else False

        graham_results.append({
            "Ticker": ticker,
            "Name": name,
            "EPS": round(eps, 2) if not np.isnan(eps) else "N/A",
            "BVPS": round(bvps, 2) if not np.isnan(bvps) else "N/A",
            "Current Price": round(price, 2) if not np.isnan(price) else "N/A",
            "Graham Number": round(graham_val, 2) if not np.isnan(graham_val) else "N/A",
            "Undervalued (Graham)": "Yes" if undervalued else "No"
        })

    except Exception as e:
        graham_results.append({
            "Ticker": ticker,
            "Name": "Error",
            "EPS": "N/A",
            "BVPS": "N/A",
            "Current Price": "N/A",
            "Graham Number": "N/A",
            "Undervalued (Graham)": f"Error: {str(e)}"
        })

#result
df_graham = pd.DataFrame(graham_results)
pd.set_option('display.float_format', '{:.2f}'.format)
display(df_graham)

,Ticker,Name,EPS,BVPS,Current Price,Graham Number,Undervalued (Graham)
0,KO,Coca-Cola Company (The),2.82,6.64,70.34,20.53,No
1,PG,Procter & Gamble Company (The),6.51,21.88,153.51,56.61,No
2,JNJ,Johnson & Johnson,9.34,32.60,173.33,82.78,No
3,JPM,JP Morgan Chase & Co.,19.49,122.51,288.76,231.79,No
4,WMT,Walmart Inc.,2.34,10.49,103.73,23.50,No
5,T,AT&T Inc.,1.75,14.70,28.08,24.06,No
6,XOM,Exxon Mobil Corporation,7.04,61.59,106.80,98.78,No
7,AXP,American Express Company,14.25,46.42,297.43,122.00,No
8,INTC,Intel Corporation,-4.77,22.36,19.95,N/A,No


 # 6.ROE （股東權益報酬率）

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np

# ✅ 抓取 ROE 與公司產業資料
def get_roe_info(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        roe = info.get("returnOnEquity", None)
        name = info.get("shortName", ticker)
        industry = info.get("industry", "N/A")
        sector = info.get("sector", "N/A")

        return {
            "Ticker": ticker,
            "Name": name,
            "Sector": sector,
            "Industry": industry,
            "ROE (%)": round(roe * 100, 2) if roe is not None else "N/A"
        }
    except Exception as e:
        return {
            "Ticker": ticker,
            "Name": "Error",
            "Sector": "N/A",
            "Industry": "N/A",
            "ROE (%)": f"Error: {str(e)}"
        }



tickers = ["AAPL", "MSFT", "KO", "PG", "JNJ", "AXP", "JPM", "WMT", "XOM"]


# ✅ 批次處理
roe_results = [get_roe_info(tkr) for tkr in tickers]
df_roe = pd.DataFrame(roe_results)
df_roe["ROE_num"] = pd.to_numeric(df_roe["ROE (%)"], errors="coerce")

# 定義分類函數
def classify_roe(roe):
    if pd.isna(roe):
        return "N/A"
    elif roe >= 20:
        return "資本效率極佳"
    elif roe >= 10:
        return "普通優秀企業"
    elif roe >= 0:
        return "資本效率偏低"
    else:
        return "虧損或異常值"

# 套用分類邏輯
df_roe["ROE 評估等級"] = df_roe["ROE_num"].apply(classify_roe)


# ✅ 排序（排除非數值欄位）
df_roe_sorted = df_roe.copy()
df_roe_sorted["ROE (%) (sort)"] = pd.to_numeric(df_roe["ROE (%)"], errors='coerce')
df_roe_sorted = df_roe_sorted.sort_values(by="ROE (%) (sort)", ascending=False).drop(columns="ROE (%) (sort)").reset_index(drop=True)

# ✅ 顯示表格
pd.set_option('display.float_format', '{:.2f}'.format)
df_roe = df_roe_sorted 
display(df_roe_sorted)


,Ticker,Name,Sector,Industry,ROE (%),ROE_num,ROE 評估等級
0,AAPL,Apple Inc.,Technology,Consumer Electronics,149.81,149.81,資本效率極佳
1,KO,Coca-Cola Company (The),Consumer Defensive,Beverages - Non-Alcoholic,42.37,42.37,資本效率極佳
2,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,33.28,33.28,資本效率極佳
3,AXP,American Express Company,Financial Services,Credit Services,32.81,32.81,資本效率極佳
4,PG,Procter & Gamble Company (The),Consumer Defensive,Household & Personal Products,31.24,31.24,資本效率極佳
5,JNJ,Johnson & Johnson,Healthcare,Drug Manufacturers - General,30.21,30.21,資本效率極佳
6,WMT,Walmart Inc.,Consumer Defensive,Discount Stores,21.78,21.78,資本效率極佳
7,JPM,JP Morgan Chase & Co.,Financial Services,Banks - Diversified,16.21,16.21,普通優秀企業
8,XOM,Exxon Mobil Corporation,Energy,Oil & Gas Integrated,11.83,11.83,普通優秀企業


# 7. EV / EBITDA 估值模型

In [12]:
import yfinance as yf
import pandas as pd
import numpy as np

def calculate_ev_ebitda(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        name = info.get("shortName", ticker)
        sector = info.get("sector", "N/A")
        industry = info.get("industry", "N/A")

        market_cap = info.get("marketCap", np.nan)
        debt = info.get("totalDebt", 0)
        cash = info.get("totalCash", 0)
        ebitda = info.get("ebitda", np.nan)

        # 檢查是否可計算 EV/EBITDA
        if all([not np.isnan(val) for val in [market_cap, ebitda]]) and ebitda != 0:
            ev = market_cap + debt - cash
            ev_ebitda = ev / ebitda

            # 標記是否屬於合理區間（常見合理值為 6–15）
            if ev_ebitda < 6:
                tag = "possibly undervalued"
            elif ev_ebitda > 15:
                tag = "possibly overvalued"
            else:
                tag = "Reasonable"

            return {
                "Ticker": ticker,
                "Name": name,
                "Sector": sector,
                "Industry": industry,
                "Market Cap (B)": round(market_cap / 1e9, 2),
                "EBITDA (B)": round(ebitda / 1e9, 2),
                "EV/EBITDA": round(ev_ebitda, 2),
                "估值評估": tag
            }
        else:
            return {
                "Ticker": ticker,
                "Name": name,
                "Sector": sector,
                "Industry": industry,
                "Market Cap (B)": "N/A",
                "EBITDA (B)": "N/A",
                "EV/EBITDA": "N/A",
                "估值評估": "N/A"
            }

    except Exception as e:
        return {
            "Ticker": ticker,
            "Name": "Error",
            "Sector": "N/A",
            "Industry": "N/A",
            "Market Cap (B)": "N/A",
            "EBITDA (B)": "N/A",
            "EV/EBITDA": "N/A",
            "估值評估": str(e)}

            
# 股票清單
tickers = ["AAPL", "MSFT", "KO", "JNJ", "T", "WMT", "XOM", "JPM", "META"]


# 執行估值計算
ev_ebitda_results = [calculate_ev_ebitda(tkr) for tkr in tickers]
df_ev = pd.DataFrame(ev_ebitda_results)
df_ev["EV/EBITDA_num"] = pd.to_numeric(df_ev["EV/EBITDA"], errors="coerce")


def classify_ev_ebitda(ev_val):
    if pd.isna(ev_val):
        return "N/A"
    elif ev_val < 6:
        return "underestimated（可能低成長）"
    elif ev_val > 15:
        return "overestimated（可能高成長）"
    else:
        return "reasonable"


df_ev["EV/EBITDA 區間評價"] = df_ev["EV/EBITDA_num"].apply(classify_ev_ebitda)
df_ev.drop(columns=["EV/EBITDA_num"], inplace=True)
pd.set_option('display.float_format', '{:.2f}'.format)
display(df_ev)

,Ticker,Name,Sector,Industry,Market Cap (B),EBITDA (B),EV/EBITDA,估值評估,EV/EBITDA 區間評價
0,AAPL,Apple Inc.,Technology,Consumer Electronics,3403.65,141.70,24.35,possibly overvalued,overestimated（可能高成長）
1,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,3880.41,156.53,24.90,possibly overvalued,overestimated（可能高成長）
2,KO,Coca-Cola Company (The),Consumer Defensive,Beverages - Non-Alcoholic,302.72,15.79,21.44,possibly overvalued,overestimated（可能高成長）
3,JNJ,Johnson & Johnson,Healthcare,Drug Manufacturers - General,417.44,30.25,14.85,Reasonable,reasonable
4,T,AT&T Inc.,Communication Services,Telecom Services,200.78,44.61,7.67,Reasonable,reasonable
5,WMT,Walmart Inc.,Consumer Defensive,Discount Stores,827.81,42.76,20.75,possibly overvalued,overestimated（可能高成長）
6,XOM,Exxon Mobil Corporation,Energy,Oil & Gas Integrated,455.32,62.38,7.69,Reasonable,reasonable
7,JPM,JP Morgan Chase & Co.,Financial Services,Banks - Diversified,N/A,N/A,N/A,N/A,N/A
8,META,"Meta Platforms, Inc.",Communication Services,Internet Content & Information,1932.59,94.28,20.52,possibly overvalued,overestimated（可能高成長）


# Conclusion

In [8]:
import pandas as pd
import numpy as np

# 安全取得模型欄位值
def safe_get(df, ticker, column):
    try:
        return df[df["Ticker"] == ticker][column].values[0]
    except:
        return "不適用"

# 整合函式
def collect_model_estimates(df_dcf, df_ddm, df_buffett, df_graham, df_relative, df_roe, df_ev):
    # 整合所有股票
    tickers = sorted(set(
        df_dcf["Ticker"]) |
        set(df_ddm["Ticker"]) |
        set(df_buffett["Ticker"]) |
        set(df_graham["Ticker"]) |
        set(df_relative["Ticker"]) |
        set(df_roe["Ticker"]) |
        set(df_ev["Ticker"])
    )

    summary = []

    for ticker in tickers:
        row = {"Ticker": ticker}

        # DCF
        note = safe_get(df_dcf, ticker, "Note")
        row["DCF"] = "低估" if note == "Undervalued" else "高估" if note == "Overvalued" else "不適用"

        # DDM
        note = safe_get(df_ddm, ticker, "Note")
        row["DDM"] = "低估" if note == "Undervalued" else "高估" if note == "Overvalued" else "不適用"

        # Buffett
        val = safe_get(df_buffett, ticker, "Undervalued (Buffett)")
        row["Buffett"] = "低估" if val == "Yes" else "高估" if val == "No" else "不適用"

        # Graham Number
        val = safe_get(df_graham, ticker, "Undervalued (Graham)")
        row["Graham Number"] = "低估" if val == "Yes" else "高估" if val == "No" else "不適用"

        # P/E, P/B
        pe = safe_get(df_relative, ticker, "P/E Undervalued")
        pb = safe_get(df_relative, ticker, "P/B Undervalued")
        if pd.isna(pe) and pd.isna(pb):
            row["P/E、P/B 相對估值"] = "不適用"
        elif pe or pb:
            row["P/E、P/B 相對估值"] = "低估"
        else:
            row["P/E、P/B 相對估值"] = "高估"

        # ROE 評估模型
        row["ROE 評估模型"] = safe_get(df_roe, ticker, "ROE 評估等級")

        # EV/EBITDA 倍數法
        row["EV/EBITDA 倍數法"] = safe_get(df_ev, ticker, "估值評估")

        summary.append(row)

    return pd.DataFrame(summary)

# ⬇️ 使用範例
df_summary = collect_model_estimates(df_dcf, df_ddm, df_buffett, df_graham, df_relative, df_roe, df_ev)


# 分數對應表
score_map = {
    "低估": 1,
    "高估": -1,
    "可能高估": -1,
    "資本效率極佳": 1,
    "普通優秀企業": 0,
    "合理區間": 0,
    "不適用": 0,
    np.nan: 0
}

# 需要評分的模型欄位
model_cols = [
    "DCF", "DDM", "Buffett", "Graham Number",
    "P/E、P/B 相對估值", "ROE 評估模型", "EV/EBITDA 倍數法"
]

# 計算每一欄的分數
for col in model_cols:
    df_summary[col + "_分數"] = df_summary[col].map(score_map).fillna(0)

# 加總有效模型數（只排除 "不適用" 與 NaN）
df_summary["有效模型數"] = df_summary[model_cols].apply(
    lambda row: sum([1 for v in row if v not in ["不適用", np.nan]]), axis=1
)

# 加總總得分
df_summary["估值總得分"] = df_summary[[col + "_分數" for col in model_cols]].sum(axis=1)

# 計算最終標準化得分（-1～+1）
df_summary["Score"] = (df_summary["估值總得分"] / df_summary["有效模型數"]).round(2)

# 👉 清理中間欄位（可選）
df_summary.drop(columns=[col + "_分數" for col in model_cols] + ["估值總得分"], inplace=True)

# ✅ 顯示最終結果
display(df_summary.sort_values("Score", ascending=False).reset_index(drop=True))



,Ticker,DCF,DDM,Buffett,Graham Number,P/E、P/B 相對估值,ROE 評估模型,EV/EBITDA 倍數法,有效模型數,Score
0,AMZN,不適用,不適用,不適用,不適用,低估,不適用,不適用,1,1.00
1,AVGO,不適用,不適用,不適用,不適用,低估,不適用,不適用,1,1.00
2,CRM,不適用,不適用,不適用,不適用,低估,不適用,不適用,1,1.00
3,GOOG,低估,不適用,不適用,不適用,低估,不適用,不適用,2,1.00
4,TMUS,不適用,低估,不適用,不適用,低估,不適用,不適用,2,1.00
5,XOM,低估,低估,不適用,高估,低估,普通優秀企業,合理區間,6,0.33
6,TSM,低估,不適用,高估,不適用,低估,不適用,不適用,3,0.33
7,T,不適用,低估,不適用,高估,低估,不適用,合理區間,4,0.25
8,META,不適用,不適用,不適用,不適用,低估,不適用,可能高估,2,0.00
9,JNJ,高估,低估,高估,高估,低估,資本效率極佳,合理區間,7,0.00
